In [43]:
# jump to input 234 to pick up from previous file
# add imports for decoding json and regex
import json
import re

import requests
import scrapy



In [44]:
# Adds the User-Agent becase goheels blockes the python agent.
url = "http://goheels.com/roster.aspx?path=mbball"
headers = {'User-Agent': 'UNC Journo Class'}
resp = requests.get(url, headers={'User-Agent': headers},)


InvalidHeader: Value for header {User-Agent: {'User-Agent': 'UNC Journo Class'}} must be of type str or bytes, not <class 'dict'>

In [ ]:
body_bytes = resp.content

body_str = body_bytes.decode('utf-8')
body_str

In [ ]:
sel = scrapy.Selector(text=body_str)

In [ ]:
#selects the first table using [index]
table = sel.css('.sidearm-table')[0]
table

In [ ]:
table.css('tr') # Get all the tr elements within the table
table.css('tr')[0] # Get the first tr element within the table using the pythonic index
table.css('tr')[0].css('th') # Get all the th elements from within the first tr
table.css('tr:nth-child(1) th') # Get all the th elements from the first tr element within the table using CSS selector
# !!!! Note the table on the page has a header and a footer and there will be TWO sets of tr's with th's

In [ ]:
table.css('tr:nth-child(1) th')

In [ ]:
table.css('tr')[0].css('th').xpath('string()') 

In [ ]:
table.css('tr')[0].css('th').xpath('string()').extract()

In [ ]:
col_names = table.css('tr:nth-child(1) th').xpath('string()').extract()

In [ ]:
col_names

In [ ]:
other_rows = table.css('tr:not(:nth-child(1))')

In [ ]:
other_rows

In [ ]:
# my starting point for just getting the information from the rows
by_player_num = {}
for row in other_rows:
    player_num = row.css('td')[0].xpath('string()').extract()[0]
    player_name = row.css('td')[1].xpath('string()').extract()[0]
    player_position = row.css('td')[2].xpath('string()').extract()[0]
    player_num_data = []
    player_name_data = []
    
    by_player_num[player_num] = player_name, player_position 


In [ ]:
by_player_num

In [ ]:

#populate all the data to the object organized by number
by_player_num = {}
player_data = []
for row in other_rows:
    
    player_num = row.css('td')[0].xpath('string()').extract()[0]
    player_name = row.css('td')[1].xpath('string()').extract()[0]
    # get the second cell, then the link, then just the href
    link = row.css('td')[1]
    link = link.css('a')
    player_href= link.xpath('@href').extract()
    player_position = row.css('td')[2].xpath('string()').extract()[0]
    player_num_data = []
    player_data = {
        "name": player_name,
        "position": player_position,
        "href": player_href[0],
    }
    
    
    
#     player_data.append({
#             "name": player_name,
#             "position": player_position,
#             "href": player_href[0],
#         })
    
    by_player_num[player_num] = player_data 
   

In [47]:
by_player_num

{'1': {'href': '/roster.aspx?rp_id=13515',
  'image': '/images/2017/9/12/Pinson_Theo_mb_2017_23.jpg?width=300',
  'name': 'Theo Pinson',
  'position': 'F/G'},
 '11': {'href': '/roster.aspx?rp_id=13518',
  'image': '/images/2017/12/22/Rush_Shea.jpg?width=300',
  'name': 'Shea Rush',
  'position': 'F'},
 '13': {'href': '/roster.aspx?rp_id=13525',
  'image': '/images/2017/9/12/Johnson_Cameron_mb_2017_130.jpg?width=300',
  'name': 'Cameron Johnson',
  'position': 'G'},
 '14': {'href': '/roster.aspx?rp_id=14170',
  'image': '/images/2017/12/22/Ma_Kane.jpg?width=300',
  'name': 'Kane Ma',
  'position': 'G'},
 '15': {'href': '/roster.aspx?rp_id=13522',
  'image': '/images/2017/9/12/Brooks_Garrison_mb_2017_221.jpg?width=300',
  'name': 'Garrison Brooks',
  'position': 'F'},
 '2': {'href': '/roster.aspx?rp_id=13508',
  'image': '/images/2017/9/12/Berry_Joel_mb_2017_59.jpg?width=300',
  'name': 'Joel Berry II',
  'position': 'G'},
 '21': {'href': '/roster.aspx?rp_id=13526',
  'image': '/images/2

In [45]:
# Create a regular expression to find javascript objects on the page being passed to a function call...

# There is JS on the page that looks like
# $.getJSON("/services/responsive-roster-bio.ashx", { type: 'stats', rp_id: 9450, path: 'baseball', year: 2017, player_id: 3759 }
# and we want the object:
# { type: 'stats', rp_id: 9450, path: 'baseball', year: 2017, player_id: 3759 }
# This pattern will match and capture anything between curly braces following the ashx url
js_obj_rx = re.compile(r'.*?responsive-roster-bio\.ashx.*?({.*?})')



# now get data from secondary, profile page
for player in list(by_player_num.values()):
    new_url = 'http://goheels.com' + player["href"]
    resp = requests.get(new_url, headers={'User-Agent': 'UNC Journo Class'},)
    content = resp.content.decode('utf-8')
    sel = scrapy.Selector(text=content)
    #selects the image url
    player_image = sel.css('.sidearm-roster-player-image img').xpath("@src").extract()[0]
    # selects the biography
    player_bio = sel.css('#sidearm-roster-player-bio').xpath('string()').extract()[0]

    player["image"] = player_image
    #player["bio"] = player_bio
    
    # Now we know what leads up to the content we're looking for so 
    # we split the text content some token that preceeds our 
    # regex to reduce the text size
    parts = content.split('$.getJSON("/services/')[1:]
    
    # Now we run our regular expression on the joined 
    # parts (put them back in to a single string)
    # and capture the JS objects
    captured = js_obj_rx.findall(''.join(parts))
    
    # Turn it in to valid json by splitting everything apart
    clean_objs = []
    for obj_str in captured:
        # We only want the stats object...
        if 'stats' not in obj_str:
            continue

        # Remove the braces
        obj_str = obj_str.replace('{', '').replace('}', '')
        # Remove the quotes
        obj_str = obj_str.replace("'", '').replace('"', '')
        # Split apart on commas
        obj_pairs = obj_str.split(',')
        # Now it looks like [" type: related", " rp_id: 9450", ...]
        # So we need to wrap everything in quotes to make it json so lets get separate values
        obj_pairs = [x.split(":") for x in obj_pairs]
        # Now it looks like [(' type', ' related'), ]
        # and we need to clean up all the leading / trailing white space
        # and wrap each value in double quotes
        clean_pairs = []
        for pair in obj_pairs:
            clean_pairs.append(['"{}"'.format(p.strip()) for p in pair])
        # And join them together with colons
        colonized = [":".join(p) for p in clean_pairs]
        # So now colonized is ['"type":"stats"']
        # and we need to join those together with commas
        commas = ','.join(colonized)
        # And make it look like a JSON object...
        json_str = "{" + commas + "}"
        # And make it a python dictionary
        clean_objs.append(json.loads(json_str))
        
    # That was a lot of work! but now we can go get stats directly
    #double splat
    #keyword arg
    stats_url = (
        "http://goheels.com/services/responsive-roster-bio.ashx?"
        "type={type}&rp_id={rp_id}&path={path}&year={year}"
        "&player_id={player_id}"
    ).format(**clean_objs[0])
    
    
    resp = requests.get(stats_url, headers=headers)
    sel = scrapy.Selector(text=resp.content.decode('utf-8'))
    
    html = json.loads(resp.content.decode("utf-8"))["current_stats"]
    sel = scrapy.Selector(text=html)



In [46]:
#clean_objs
#stats_url
sel

# That looks weird... look closely at the data... html... body ... p... JSON!
#resp.content


<Selector xpath=None data='<html><body><section>\r\n        <h5>Game-'>